# Scikit-Learn IRIS Model Using Flatbuffers Protocol (Experimental)

 * Wrap a scikit-learn python model using the Flatbuffers low level low-latency protocol.
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Depenencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * [Flatbuffers](https://google.github.io/flatbuffers/)

```bash
pip install sklearn
pip install grpcio-tools
```

### Flatbuffers Install
```
wget https://github.com/google/flatbuffers/archive/v1.9.0.tar.gz
tar -xvxf v1.9.0.tar.gz
cd flatbuffers-1.9.0/python
python setup.py install
```
## Train locally
 

In [ ]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn import datasets

def main():
    clf = LogisticRegression()
    p = Pipeline([('clf', clf)])
    print('Training model...')
    p.fit(X, y)
    print('Model trained!')

    filename_p = 'IrisClassifier.sav'
    print('Saving model in %s' % filename_p)
    joblib.dump(p, filename_p)
    print('Model saved!')
    
if __name__ == "__main__":
    print('Loading iris data set...')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print('Dataset loaded!')
    main()


Wrap model using s2i

## Flatbuffers Protocol Test

In [ ]:
!s2i build . seldonio/seldon-core-s2i-python3:0.2 sklearn-iris:0.1

In [ ]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 sklearn-iris:0.1 

In [ ]:
!cd ../../../wrappers/testing && make build_fbs

Send some random features that conform to the contract

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --fbs

In [ ]:
!docker rm iris_predictor --force